In [2]:
import torch
import gym as g
from gym import spaces
from connect4 import *
from envs import ConnectNEnv
import numpy as np

In [3]:
env = ConnectNEnv()

In [5]:
# Test
env.step(0)
env.reset()

({'observations': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        dtype=int8),
  'action_mask': array([1, 1, 1, 1, 1, 1, 1], dtype=int8)},
 {})